In [2]:
import numpy as np 
from astropy.io import fits
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib ipympl

In [3]:
'''
column 1: group ID
column 2: richness
column 3: ra
column 4: dec
column 5: redshift
column 6: log M_h/(M_sun/h)
column 7: L_group (10^10Lsun/h/h)
'''
group = np.loadtxt('../odata/CLAUDS_HSC_iband_group')

In [32]:
digal = np.loadtxt('../odata/CLAUDS_HSC_iband_igal')
d2 = np.loadtxt('../odata/iCLAUDS_HSC_iband_2')
d1 = np.loadtxt('../odata/iCLAUDS_HSC_iband_1')

## observational effects

In [44]:
idx_sel = np.where((group[:,4] > 0.8) & (group[:,4] < 1.2))[0] #select with Mh > 1e14 and z < 0.1
Ng0 = group[idx_sel,1]
print(np.median(Ng0[:100]))
plt.figure()

plt.hist(group[idx_sel,1][:100])

196.5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([33., 23.,  9., 13.,  5.,  8.,  2.,  3.,  1.,  3.]),
 array([141. , 175.2, 209.4, 243.6, 277.8, 312. , 346.2, 380.4, 414.6,
        448.8, 483. ]),
 <BarContainer object of 10 artists>)

In [35]:
def cal_Nsat(redz,redzbin):  
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    from astropy.coordinates import SkyCoord
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
        
    idx_sel = np.where((group[:,4] > redzbin[0]) & (group[:,4] < redzbin[1]))[0]
    idx_sel = idx_sel[:100] #take 100 richness groups
    clu_id = np.int64(group[idx_sel,0])
    Nc = idx_sel.shape[0]
    Nsat = np.zeros(Nc, dtype = np.int64)
    for i in range(Nc):
        idx_gal = np.where(d2[:,0] == clu_id[i])[0]
        galid = np.int64(d2[idx_gal,1])
        
        gal_abm = digal[galid-1,5]

        d_L = cosmo.luminosity_distance(redz).value * 0.674
        Mc = 26 - 5*np.log10(d_L) - 25

        Nsat[i] = np.where(gal_abm < Mc)[0].shape[0]
        
    print('z = %s: min %s, mean %s' %(redz, np.min(Nsat), np.mean(Nsat)))
    return Nsat

In [45]:
Nsat = cal_Nsat(5.4,[0.8,1.2])
print(np.median(Nsat), np.mean(Nsat))

z = 5.4: min 0, mean 2.68
2.0 2.68


In [41]:
np.mean(Nsat), np.median(Nsat)

(2.68, 2.0)

In [42]:
plt.figure()

plt.hist(Nsat)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([11., 24., 16., 18., 13., 11.,  4.,  0.,  0.,  3.]),
 array([0. , 0.9, 1.8, 2.7, 3.6, 4.5, 5.4, 6.3, 7.2, 8.1, 9. ]),
 <BarContainer object of 10 artists>)

## summary plot

In [5]:
Ng = np.array([1,2,3,5,10])
z = np.array([0,1,2,3,4,5])

for i in range(6):
    for j in range(5):
        idx = np.where((group[:,1] >= Ng[j]) & (group[:,4] >= z[i]))[0]
        if len(idx) > 1:
            Mh = group[idx,-2]
            medM = np.median(Mh)
            print('redshift z >= %s, Ng >= %s:' %(z[i], Ng[j]), medM)
        elif len(idx) == 1:
            print('redshift z >= %s, Ng >= %s:' %(z[i], Ng[j]), group[idx,-2])
    print('/n')

redshift z >= 0, Ng >= 1: 11.8605
redshift z >= 0, Ng >= 2: 12.199
redshift z >= 0, Ng >= 3: 12.3239
redshift z >= 0, Ng >= 5: 12.5023
redshift z >= 0, Ng >= 10: 12.7854
/n
redshift z >= 1, Ng >= 1: 11.8869
redshift z >= 1, Ng >= 2: 12.2697
redshift z >= 1, Ng >= 3: 12.4907
redshift z >= 1, Ng >= 5: 12.7659
redshift z >= 1, Ng >= 10: 13.146
/n
redshift z >= 2, Ng >= 1: 11.8202
redshift z >= 2, Ng >= 2: 12.2233
redshift z >= 2, Ng >= 3: 12.499
redshift z >= 2, Ng >= 5: 12.8274
redshift z >= 2, Ng >= 10: 13.2462
/n
redshift z >= 3, Ng >= 1: 11.7791
redshift z >= 3, Ng >= 2: 12.1465
redshift z >= 3, Ng >= 3: 12.3999
redshift z >= 3, Ng >= 5: 12.7853
redshift z >= 3, Ng >= 10: 13.2989
/n
redshift z >= 4, Ng >= 1: 11.8677
redshift z >= 4, Ng >= 2: 12.2151
redshift z >= 4, Ng >= 3: 12.5458
redshift z >= 4, Ng >= 5: 13.004750000000001
/n
redshift z >= 5, Ng >= 1: 11.9497
redshift z >= 5, Ng >= 2: 12.19755
redshift z >= 5, Ng >= 3: [12.7696]
/n


In [9]:
Ng = np.array([1,2,3,5,10])
z = np.array([0,1,2,3,4,5])
call = np.array(['k','b','r','orange','cyan'])

plt.figure()

for i in range(6):
    for j in range(5):
        idx = np.where((group[:,1] >= Ng[j]) & (group[:,4] >= z[i]))[0]
        if len(idx):
            if i == 0:
                plt.bar(i-0.3+j*0.15, idx.shape[0], color = call[j], 
                        width = 0.15, label = r'$N_{\rm g} \geq $ %s' %Ng[j])
            else:
                plt.bar(i-0.3+j*0.15, idx.shape[0], color = call[j], width = 0.15)
            
plt.yscale('log')

ax = plt.gca()
ax.set_xticks(np.arange(0,6,1))
labels = ['total', r'$z \geq 1$', r'$z \geq 2$' , r'$z \geq 3$', r'$z \geq 4$', r'$z \geq 5$']
ax.set_xticklabels(labels)

plt.ylabel(r'$N_{\rm grp}$')

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
Ng = np.array([1,2,3,5,10])
z = np.array([0,1,2,3,4,5])
call = np.array(['k','b','r','orange','cyan'])

plt.figure()

for i in range(6):
    for j in range(5):
        idx = np.where((group[:,1] >= Ng[j]) & (group[:,4] >= z[i]))[0]
        if len(idx) > 1:
            Mh = 10**group[idx,-2]
            medM = np.median(Mh)
            errM = np.percentile(Mh, [16,84])
            yerr0 = np.log10(medM) - np.log10(errM[0])
            yerr1 = np.log10(errM[1]) - np.log10(medM)
            xx = i-0.3+j*0.15
            if i == 0:
                plt.errorbar(xx, np.log10(medM), yerr = [[yerr0],[yerr1]],
                             marker = 'o', capsize = 2, c = call[j],
                             label = r'$N_{\rm g} \geq $ %s' %Ng[j])
            else:
                plt.errorbar(xx, np.log10(medM),
                             yerr = [[yerr0],[yerr1]],
                             marker = 'o', capsize = 2, c = call[j])
        elif len(idx) == 1:
            Mh = 10**group[idx,-2]
            plt.errorbar(i-0.3+j*0.15, np.log10(Mh),
                             marker = 'o', capsize = 2, color = call[j])
            

ax = plt.gca()
ax.set_xticks(np.arange(0,6,1))
labels = ['total', r'$z \geq 1$', r'$z \geq 2$' , r'$z \geq 3$', r'$z \geq 4$', r'$z \geq 5$']
ax.set_xticklabels(labels)

plt.ylabel(r'$\log M_{\rm h} [h^{-1} \rm M_{\odot}]$')

plt.legend(loc = 'best')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
#
idx = np.where((group[:,1] == 2) & (group[:,4] > 5))[0]
grp2mass = 10**group[idx, -2]
grp2err = np.percentile(grp2mass, [16,84])

xrange = np.arange(0,5.5,0.5)
xr = (xrange[1:] + xrange[:-1]) / 2

plt.figure()

for i in range(xrange.shape[0]-1):
    idxsel = np.where((group[:,4] >= xrange[i]) & (group[:,4] < xrange[i+1]) &
                     (group[:,-2] > np.log10(grp2err[0])) & (group[:,-2] < np.log10(grp2err[1])))[0]
    Ngsel = group[idxsel,1]
    medN = np.median(Ngsel)
    errN = np.percentile(Ngsel, [16,84])
    plt.errorbar(xr[i], medN, yerr = [[errN[0]], [errN[1]]], marker = 'o', capsize = 2, c = 'b')
    
plt.xlabel('z')
plt.ylabel(r'$N_{g}$')
plt.title('%.2f < Mh < %.2f' %(np.log10(grp2err[0]), np.log10(grp2err[1])))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '12.06 < Mh < 12.43')

In [15]:
xrange

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])